### Assumptions in Machine Learning

Every machine learning model has assumptions, indeed, that is the only way we can reasonably learn from data and make assumptions about how what we learn generalizes to the real world. 

The problem is the assumptions/difficulty tradeoff. Simpler models tend to have more naive assumptions (ie, assumptions that are not likely to be true or not likely to generalize), and more complex models (such as neural networks, convolutional networks, and recurrent neural networks) have less assumptions (and therefore, save for overfitting, are more likely to generalize better to the real world), difficult (and expensive) to build, train, and deploy. 

To understand this specifically, lets go through some models and state the assumptions they make. 


### Linear Regression & Logistic Regression

Linear regression assumes that the data can be generated via a linear function, and logistic regression assumes that the data can be linearly separated. 

![lin](http://sebastianraschka.com/images/blog/2014/kernel_pca/linear_vs_nonlinear.png)

Although we may intuitively think that assuming linearity is a very naive assumption, these are two of the most common algorithms used in the industry to make reasonable predictions for the following reasons: 

1. They are relatively easy to train, and don't require expensive & difficult research to optimize
2. They have well known theoretical gaurantees and have been shown to work well in practice for decades
3. Domain expertise can be applied to make sense of what has been learned
4. The assumptions often turn out to be mostly true, and the tradeoff we have to make to get the edge cases classified correctly aren't worth the extra work.

### Naive Bayes Classifier

Bayes Rule states that for a latent variable and an evidence variable, the posterior distribution of the latent variable is equivalent to the prior probability of the latent variable times the likelihood of the evidence given the latent variable, divided by a constant (the probability of observing the evidence). 

Specifically:

$$ p(y_i | x) = \frac{p(y_i) p(x | y_i)}{p(x)} = \frac{p(x, y_i)}{\sum_i p(x, y_i)} $$

Using the rules of probability, we can write the joint distribution as a conditional:

$$ p(x, y_i) = p(x_1 | x_2, ... x_n, y_i) * p(x_2 | x_3, ... x_n, y_i) $$

Here is where the assumption comes into play: since we can't really compute these conditional probabilities, we just define: 

$$ p(x_1 | x_2, ... x_n, y_i) = p(x_1 | y_i) $$

This assumption means that each feature in our training set is entirely dependent upon the label that it is given, and is completely independent of every other feature, given the class label.

We cal also set $ p(x_1 | y_i) $ to take on whatever probability distribution we want to (such as the Gaussian); doing this is also another assumption in our model. 

### Neural Networks
- They also assume that the data are conditionally indpendent of each other. 


We have used Neural Networks and Convolutional Neural Netoworks to remove the assumptions of linearity in our data (and for convolutional neural networks, the assumption of spatial variance being important). Now we will use Recurrent Neural Networks to remove the assumption of independence (to an extent). 

Aside: A much simpler model (that doesn't work as well, but still has produced good results) that removes this independence assumption are Hidden Markov Models.



### An Introduction to Recurrent Neural Networks in Tensorflow

Recurrent Neural Networks are models that are able to model relationships between data where each input is not  independent of the previous inputs. 

For example, consider the field of Natural Language Processing (NLP). NLP deals with all tasks involved with processing texts and obtaining information (learning) from text. It is extremely common, especially today, to solve NLP tasks with machine learning (and especially deep learning). 

Recurrent Neural networks are a popular method for solving NLP problems, including predicting the sentiment of a movie review, predicting the next word (or character) in a sequence of words or characters, and even translating text to another language. This is because they can model dependencies, and often in text a current word is dependent on the previous words (you can probably think of several examples). 

Just for a break from all this technical talk, here is some Shakespeare: 

```
PANDARUS:
Alas, I think he shall be come approached and the day
When little srain would be attain'd into being never fed,
And who is but a chain and subjects of his death,
I should not sleep.

Second Senator:
They are away this miseries, produced upon my soul,
Breaking and strongly should be buried, when I perish
The earth and thoughts of many states.

DUKE VINCENTIO:
Well, your wit is in the care of side and that.

Second Lord:
They would be ruled after this chamber, and
my fair nues begun out of the fact, to be conveyed,
Whose noble souls I'll have the heart of the wars.

Clown:
Come, sir, I will make did behold your worship.

VIOLA:
I'll drink it.
```


### The Power of Recurrent Neural Networks

I lied, the above was not really Shakespeare. It was actually a poem written by a recurrent neural network that trained on a dataset of Shakespeare's writings. This should give you an idea of the power of RNNs: just by looking at characters (not even words!) it learned how to generate strikingly similar text to Shakespeare. Granted, it does not make much sense but it is likely that at least engineers will not be able to tell the difference.

This was from an awesome article here http://karpathy.github.io/2015/05/21/rnn-effectiveness/


Recurrent Neural Networks are important because they model sequential data. Each neuron, in addition to its data input, receives a previous ** state ** which is just a vector of numbers describing some previous computation in the RNN. This state input is what allows RNNs to learn dependencies in sequential data. 

It's a lot similar to you reading this sentence - when you understand the meaning of a particular word, you don't start figuring it out from scratch, but take the previous words and sentences into account. 

Here is what an RNN looks like: 

![rnn](https://cdn-images-1.medium.com/max/1600/1*V2W4TCmTj2h1CE7I-DngPw.png)


And this is what an RNN looks like, predicting words character by character: 

![rnn-2](https://cdn-images-1.medium.com/max/1600/1*IMalbwl6uj3nlqxixZYFvA.jpeg)

To understand RNNs deeply, we'll need to get into some data and code. 

### The data

To clearly demonstrate the fundamental idea of RNNs learning dependencies in data, we will be using numerical data with clearly observable and quantifiable dependencies, and then we'll be able to check if the RNN can learn these dependencies. 

All of the code is written in a general enough format that words (represented as one-hot indices into a vocabulary or learned word vectors) can be fed into this model, however.


**Input Sequence (X)** The input sequence is just a bunch of ones and zeros generated randomly. At each index, or more appropriately for RNNs, at each time *t* $X_t$ is 1 with probability 0.5 else 0. 

**Example:** $ X = [1, 0, 1, 1, 0, 0, ...] $

**output Sequence (Y)** The output sequence will be generated from the input sequence, and our recurrent neural network will have to learn the rules by which the output sequence is generated. 

At each step in time $t$, $Y_t$ starts off as 1 with an initial probability of $0.5$. However if $X_{t-3} = 1$ then it $Y_t$ is almost surely 1 (ie, we raise the probability $p(y_t = 1)$ by 0.5 

But, if $X_{t-8}$ is also 1 then we will decrease $P(y_t = 1)$ by 0.25. 

Hopefully, it is clear that if $X_{t-3} = 1$ and $X_{t-8} = 1$ then $y_t = 1 $ with probability 0.75. 

Note that there are no rules (ie, changes in $p(y_i = 1)$ if X is 0. 

This data is pretty simple. In fact, it is so simple that we can calculate exactly what our RNN should learn (given enough iterations and data) in order to determine if it has learned no dependencies, the dependency from 3 time steps ago, and the dependency from 8 time steps ago. 

#### Network Learns no dependencies

To calculate the probability with which our network will predict $y_t = 1$, we can look at our generated data. 

First, in our generated data, $y_t = 1$ with initial probability 0.5. But we added 0.5 if $x_{t-3} = 1$, which itself has a probability $0.5$. So now, $p(y_t = 1) = 0.5 + 0.5^2 = 0.75$. However, we also need to remove .25 if $x_{t-8} = 1$, which has a probability of $0.5$. 

So in our generated data, $p(y_t = 1) = 0.5 + 0.5^2 - 0.25*0.5 = 0.625 $

Since our network does not know any conditional dependence on our input and output data, it will just learn the distribution assuming that $y_t$ is independent of all $X$. So our network will assign every $y_t$ a probability of 0.625.

The cross entropy loss that corresponds to this is $ -(.625 * log(.625) + (1 - .625) * log(1 - .625)) = 0.66$. 

#### Network Learns one dependency

If our network realizes that whenever $x_{t-3} = 1$, the probability $p(y_t) = 1$ goes up by 0.5, then it will assign a probability of $0.875$ when $x_{t-3} = 1$, because $p(y_t = 1) = 0.625 + 0.5*.5 = 0.875$ and a probability of $0.625$ when $x_{t-3} = 1$. 

The corresponding cross entropy loss is $ -0.5 * (0.875 * log(0.875) + 0.125 * log(0.125)) + -0.5 * (0.625 * log(0.625) + 0.375 * log(0.375)))  = 0.52$

#### Neural Network learns both dependencies

Exercise: work out the probabilities with which our network will assign $p(y_t = 1)$, and calculate the cross-entropy. 

To check your answer, the cross entropy you should get is 0.45. 


So now we know that if we train our network correctly: 

- With one hidden state, we should learn no dependencies, so our final loss should be 0.66. 

- With three or more hidden states, we should learn the first dependency, so our final loss should be 0.52. 

- With eight or more hidden states, we should learn both dependencies, so our final loss should be 0.46. 

Let's start building our RNN to see if our hypothesis is supported. 




In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from gen_data import gen_data # gets the data for this project

In [2]:
RNNconfig = {
    'num_steps' : 5, # higher n = capture longer term dependencies, but more expensive (and potential vanishing gradient issues)
    'batch_size' : 200,
    'state_size' :4,
    'learning_rate' : 0.1
}

num_classes = 2


In [3]:
# adapted from https://github.com/tensorflow/tensorflow/blob/master/tensorflow/models/rnn/ptb/reader.py
def gen_batch(raw_data, batch_size, num_steps):
    raw_x, raw_y = raw_data
    data_length = len(raw_x)

    # partition raw data into batches and stack them vertically in a data matrix
    batch_partition_length = data_length // batch_size
    data_x = np.zeros([batch_size, batch_partition_length], dtype=np.int32)
    data_y = np.zeros([batch_size, batch_partition_length], dtype=np.int32)
    for i in range(batch_size):
        data_x[i] = raw_x[batch_partition_length * i:batch_partition_length * (i + 1)]
        data_y[i] = raw_y[batch_partition_length * i:batch_partition_length * (i + 1)]
    # further divide batch partitions into num_steps for truncated backprop
    epoch_size = batch_partition_length // num_steps

    for i in range(epoch_size):
        x = data_x[:, i * num_steps:(i + 1) * num_steps]
        y = data_y[:, i * num_steps:(i + 1) * num_steps]
        yield (x, y)

def gen_epochs(n, num_steps):
    for i in range(n):
        yield gen_batch(gen_data(), RNNconfig['batch_size'], num_steps) # python trivia: why do we use yield instead of return here?

In [4]:

# placeholder
x = tf.placeholder(tf.int32, [RNNconfig['batch_size'], RNNconfig['num_steps']], name='input_placeholder')
y = tf.placeholder(tf.int32, [RNNconfig['batch_size'], RNNconfig['num_steps']], name='labels_placeholder')
init_state = tf.zeros([RNNconfig['batch_size'], RNNconfig['state_size']])

"""
RNN Inputs
"""

# Turn our x placeholder into a list of one-hot
# rnn_inputs is a list of num_steps tensors with shape [batch_size, num_classes]
x_one_hot = tf.one_hot(x, num_classes) # note: num_classes is not an RNN variable...
rnn_inputs = tf.unstack(x_one_hot, axis=1)

In [5]:
"""
from __call__ method
https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/rnn/python/ops/rnn_cell.py
"""
# with tf.variable_scope('rnn_cell'):
#     W = tf.get_variable('W', [num_classes + RNNconfig['state_size'], RNNconfig['state_size']])
#     b = tf.get_variable('b', [RNNconfig['state_size']], initializer=tf.constant_initializer(0.0))

# def rnn_cell(rnn_input, state):
#     with tf.variable_scope('rnn_cell', reuse=True):
#         W = tf.get_variable('W', [num_classes + RNNconfig['state_size'], RNNconfig['state_size']])
#         b = tf.get_variable('b', [RNNconfig['state_size']], initializer=tf.constant_initializer(0.0))
#     return tf.tanh(tf.matmul(tf.concat([rnn_input, state], 1), W) + b)

'\nfrom __call__ method\nhttps://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/rnn/python/ops/rnn_cell.py\n'

In [ ]:
"""
https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/rnn/python/ops/rnn_cell.py
"""
# state = init_state
# rnn_outputs = []
# for rnn_input in rnn_inputs:
#     state = rnn_cell(rnn_input, state)
#     rnn_outputs.append(state)
# final_state = rnn_outputs[-1]
cell = tf.contrib.rnn.BasicLSTMC(RNNconfig['state_size'])

rnn_outputs, final_state = tf.contrib.rnn.static_rnn(cell, rnn_inputs, initial_state = init_state)

In [ ]:
"""
https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/seq2seq/python/ops/loss.py#L30
"""

#logits and predictions
with tf.variable_scope('softmax'):
    W = tf.get_variable('W', [RNNconfig['state_size'], num_classes])
    b = tf.get_variable('b', [num_classes], initializer=tf.constant_initializer(0.0))
logits = [tf.matmul(rnn_output, W) + b for rnn_output in rnn_outputs]
predictions = [tf.nn.softmax(logit) for logit in logits]

# Turn our y placeholder into a list of labels
y_as_list = tf.unstack(y, num=RNNconfig['num_steps'], axis=1)

#losses and train_step
losses = [tf.nn.sparse_softmax_cross_entropy_with_logits(labels=label, logits=logit) for \
          logit, label in zip(logits, y_as_list)]
total_loss = tf.reduce_mean(losses)
train_step = tf.train.AdagradOptimizer(0.1).minimize(total_loss)

In [ ]:

def train_network(num_epochs, verbose=True):
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        training_losses = []
        for idx, epoch in enumerate(gen_epochs(num_epochs, RNNconfig['num_steps'])):
            training_loss = 0
            training_state = np.zeros((RNNconfig['batch_size'], RNNconfig['state_size']))
            if verbose:
                print("\nEPOCH", idx)
            for step, (X, Y) in enumerate(epoch):
                tr_losses, training_loss_, training_state, _ = \
                    sess.run([losses,
                              total_loss,
                              final_state,
                              train_step],
                                  feed_dict={x:X, y:Y, init_state:training_state})
                training_loss += training_loss_
                if step % 100 == 0 and step > 0:
                    if verbose:
                        print("Average loss at step", step,
                              "for last 100 steps:", training_loss/100)
                    training_losses.append(training_loss/100)
                    training_loss = 0

    return training_losses

In [ ]:

training_losses = train_network(num_epochs = 5)
plt.plot(training_losses)

### Sources (and additional links for reading): 

1. https://r2rt.com/recurrent-neural-networks-in-tensorflow-i.html
2. http://karpathy.github.io/2015/05/21/rnn-effectiveness/
3. http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/